In [1]:
%pip install Faker psycopg2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
from faker import Faker
import psycopg2

print("Libraries installed and working!")


Libraries installed and working!


In [3]:
import psycopg2
from faker import Faker
from datetime import datetime
import random

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="restaurant_db",
    user="postgres",
    password="chinmay"  # ⬅️ Replace with your actual password
)

cur = conn.cursor()
print("Connection successful ✅")


Connection successful ✅


In [4]:
# Create Faker instance
fake = Faker()


In [5]:
for _ in range(10):
    name = fake.name()
    phone = fake.phone_number()
    email = fake.email()

    cur.execute("""
        INSERT INTO customers (name, phone, email)
        VALUES (%s, %s, %s)
    """, (name, phone, email))

conn.commit()
print("Inserted fake customers ✅")


Inserted fake customers ✅


In [6]:
menu_items = [
    ("Margherita Pizza", "Pizza", 8.99),
    ("Pepperoni Pizza", "Pizza", 9.99),
    ("Veggie Burger", "Burger", 7.49),
    ("Cheeseburger", "Burger", 7.99),
    ("Caesar Salad", "Salad", 5.99),
    ("Greek Salad", "Salad", 6.49),
    ("Pasta Alfredo", "Pasta", 10.99),
    ("Pasta Marinara", "Pasta", 9.49)
]

for item in menu_items:
    cur.execute("""
        INSERT INTO menu_items (item_name, category, price)
        VALUES (%s, %s, %s)
    """, item)

conn.commit()
print("Inserted menu items ✅")


Inserted menu items ✅


In [7]:
# Get all customer IDs and menu item IDs
cur.execute("SELECT customer_id FROM customers")
customer_ids = [row[0] for row in cur.fetchall()]

cur.execute("SELECT menu_item_id, price FROM menu_items")
menu_items_data = cur.fetchall()

for customer_id in customer_ids:
    for _ in range(random.randint(2, 5)):  # 2–5 orders per customer
        order_date = fake.date_between(start_date='-3M', end_date='today')
        num_items = random.randint(1, 3)
        selected_items = random.sample(menu_items_data, num_items)

        total_amount = sum([item[1] for item in selected_items])

        # Insert order
        cur.execute("""
            INSERT INTO orders (customer_id, order_date, total_amount)
            VALUES (%s, %s, %s)
            RETURNING order_id
        """, (customer_id, order_date, total_amount))

        order_id = cur.fetchone()[0]

        # Insert order_items
        for item in selected_items:
            menu_item_id = item[0]
            quantity = random.randint(1, 3)

            cur.execute("""
                INSERT INTO order_items (order_id, menu_item_id, quantity)
                VALUES (%s, %s, %s)
            """, (order_id, menu_item_id, quantity))

conn.commit()
print("Inserted orders & order_items ✅")


Inserted orders & order_items ✅


In [8]:
cur.close()
conn.close()
print("All done! Connection closed 🔒")


All done! Connection closed 🔒
